In [ ]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

#딥러닝 라이브러리
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64') #이 부분은 에러를 방지하기 위함임 중요코드 x

In [ ]:
train = pd.read_csv('eletrain.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission3.csv')

60개의 건물, 총 85일, 24시간

In [ ]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1,1
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1,1
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1,1
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1,1


60개 건물 7일씩 24시간인데 3시간마다 관측값 있음 + 비전기냉방설비운영, 태양광 보유 NaN

In [ ]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0,0
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0,0
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0,0
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0,0
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1,1
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1,1
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1,1
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1,1


LSTM 전력사용량만 변수로 사용

In [ ]:
#전력사용량 정규화
mini = train.iloc[:, 2].min()
size = train.iloc[:,2].max() - train.iloc[:, 2].min()
train.iloc[:,2] = (train.iloc[:,2]-mini)/size 
# 다른 값도 정규화 해보기
mini = train.iloc[:, 3].min()
size = train.iloc[:,3].max() - train.iloc[:, 3].min()
train.iloc[:,3] = (train.iloc[:,3]-mini)/size 
mini = train.iloc[:, 5].min()
size = train.iloc[:,5].max() - train.iloc[:, 5].min()
train.iloc[:,5] = (train.iloc[:,5]-mini)/size 

In [ ]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,0.461072,0.257937,2.5,0.901235,0.8,0.0,0,0
1,1,2020-06-01 01,0.458624,0.261905,2.9,0.888889,0.3,0.0,0,0
2,1,2020-06-01 02,0.457017,0.253968,3.2,0.888889,0.0,0.0,0,0
3,1,2020-06-01 03,0.453729,0.238095,3.2,0.888889,0.0,0.0,0,0
4,1,2020-06-01 04,0.453437,0.234127,3.3,0.901235,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,0.231936,0.662698,2.3,0.604938,0.0,0.7,1,1
122396,60,2020-08-24 20,0.224119,0.642857,1.2,0.641975,0.0,0.0,1,1
122397,60,2020-08-24 21,0.201373,0.642857,1.8,0.641975,0.0,0.0,1,1
122398,60,2020-08-24 22,0.185982,0.634921,1.8,0.679012,0.0,0.0,1,1


In [ ]:
input_window = 996 #임의의 수
output_window = 24 
window = 12 #window는 12시간 마다
num_features = 3 #나는 특성(전력, 기온, 습도) 사용
num_power = 60 # 건물 수
end_ = 168
lstm_units = 64 #베이스라인은 32인데 난 64
dropout = 0.25 #베이스라인은 0.2 보통 0.2~0.3 사이로 하길래 0.25해봄
EPOCH = 40 #대충 40
BATCH_SIZE = 128

In [ ]:
train_x = tf.reshape(train.iloc[:,[2,3,5]].values, [num_power, 24*85, num_features])

In [ ]:
print(f'train_x.shape:{train_x.shape}')

train_x.shape:(60, 2040, 3)


In [ ]:
train_window_x= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, input_window, num_features)) 
train_window_y= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, output_window, num_features))
print(f'train_window_x.shape:{train_window_x.shape}')
print(f'train_window_y.shape:{train_window_y.shape}')

train_window_x.shape:(60, 85, 996, 3)
train_window_y.shape:(60, 85, 24, 3)


In [ ]:
train_x.shape[1]-(input_window + output_window)
#window를 왜 12시간으로 설정했을까?

1020

In [ ]:
for example in range(train_x.shape[0]):
    
    for start in range(0, train_x.shape[1]-(input_window+output_window), window):
        end=start+input_window
        train_window_x[example, start//window, :] = train_x[example, start: end               , :]
        train_window_y[example, start//window, :] = train_x[example, end  : end+ output_window, :]

train_window_x에 train 값 채우고
lstm에 넣기 위해 new_train_x를 만들고 reshape

In [ ]:
new_train_x = tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y = tf.reshape(train_window_y, [-1, output_window, num_features])
print(f'new_train_x.shape:{new_train_x.shape}')
print(f'new_train_y.shape:{new_train_y.shape}')

new_train_x.shape:(5100, 996, 3)
new_train_y.shape:(5100, 24, 3)


여기서부터는 밑에서 여러 방법 해볼거임

In [ ]:
#층 만들기
model = Sequential([LSTM(lstm_units, return_sequences=False, recurrent_dropout=dropout), Dense(output_window*num_features, kernel_initializer = tf.initializers.zeros()), Reshape([output_window, num_features])])

In [ ]:
model.compile(optimizer = 'rmsprop', loss='mae', metrics=['mae'])#Mean absolute error
save_best_only = tf.keras.callbacks.ModelCheckpoint(filepath='lstm_model.h5', monitor='val_loss', save_best_only=True)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience=5)


In [ ]:
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0: print('')
        print('.', end='')
history = model.fit(new_train_x, new_train_y, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split=0.2, verbose=0, callbacks=[PrintDot(), early_stop, save_best_only, reduceLR])


..........
..........
..........
..........

In [ ]:
prediction = np.zeros((num_power, end_, num_features))
new_test_x = train_x

for i in range(end_//output_window):
  start_ = i*output_window
  next_ = model.predict(new_test_x[:, -input_window:, :])
  new_test_x = tf.concat([new_test_x, next_], axis=1)
  print(new_test_x.shape)
  prediction[:, start_:start_ + output_window,:] = next_


2/2 [==============================] - 1s 511ms/step
(60, 2064, 3)
2/2 [==============================] - 1s 512ms/step
(60, 2088, 3)
2/2 [==============================] - 1s 510ms/step
(60, 2112, 3)
2/2 [==============================] - 2s 1s/step
(60, 2136, 3)
2/2 [==============================] - 2s 997ms/step
(60, 2160, 3)
2/2 [==============================] - 1s 524ms/step
(60, 2184, 3)
2/2 [==============================] - 1s 521ms/step
(60, 2208, 3)


test 파일을 써보자


In [ ]:
prediction = np.zeros((num_power, end_, num_features))

In [ ]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0,0
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0,0
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0,0
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0,0
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1,1
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1,1
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1,1
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1,1


In [ ]:
test = pd.read_csv('test.csv')
for i in range(0,10078,3):
  test.loc[i+1,'기온(°C)']=test.loc[i]['기온(°C)']
  test.loc[i+2,'기온(°C)']=test.loc[i]['기온(°C)']
  test.loc[i+1,'습도(%)']=test.loc[i]['습도(%)']
  test.loc[i+2,'습도(%)']=test.loc[i]['습도(%)']
test.fillna(0)
test=tf.reshape(test.iloc[:,[2,4,7]].values, [num_power, 24*7, 3])#기온, 습도
prediction = model.predict(test)

2/2 [==============================] - 0s 129ms/step


In [ ]:
for i in range(3):
  mini = train.iloc[:, 2+i].min()
  size = train.iloc[:,2+i].max() - train.iloc[:, 2+i].min()
  prediction[:,:,i] = prediction[:,:,i]*size + mini #다시 원래값 복구
  prediction[:,:,i]

In [ ]:
train = pd.read_csv('eletrain.csv')
prediction[:,:,0]

array([[18407.82722145, 18705.53761851, 18714.17889984, ...,
        15204.80084454, 17316.33305833, 17495.92022255],
       [18288.17464489, 18582.32327607, 18591.37526541, ...,
        15104.60388598, 17188.5685058 , 17378.07499007],
       [18398.93322633, 18694.89586957, 18703.34166304, ...,
        15189.72063334, 17303.10544196, 17482.07972422],
       ...,
       [18353.27278771, 18649.05642059, 18658.61292435, ...,
        15187.77085503, 17271.16650747, 17446.79379251],
       [17598.95741945, 17874.55834288, 17867.09639734, ...,
        14372.58637047, 16481.7058008 , 16679.81953667],
       [18261.02815197, 18555.96332708, 18567.55380802, ...,
        15204.90634592, 17223.17201906, 17379.75194004]])

In [ ]:
train = pd.read_csv('eletrain.csv')
for i in range(3):
  mini = train.iloc[:, 2+i].min()
  size = train.iloc[:,2+i].max() - train.iloc[:, 2+i].min()
  prediction[:,:,i] = prediction[:,:,i]*size + mini #다시 원래값 복구
  prediction[:,:,i]

In [ ]:
prediction[:,:,0]

array([[7559.34457403, 7521.73785031, 7476.05487758, ..., 4367.36776615,
        4148.71507039, 4263.07360399],
       [ 963.41452662,  938.79376215,  929.40789668, ...,  623.94173775,
         613.04655434,  619.79981744],
       [2916.64713952, 2893.19854161, 2869.84893798, ...,  827.87132326,
         832.73969325,  854.56555094],
       ...,
       [ 569.04398527,  535.28349026,  531.25663749, ...,  523.61665009,
         491.44531075,  497.89510483],
       [ 719.52418632,  666.47547082,  649.63947687, ...,  519.40662474,
         475.99064719,  486.12776338],
       [2839.82124526, 2798.02141306, 2776.29778413, ...,  853.06311297,
         826.04167518,  857.35599842]])

In [ ]:
prediction[:,:,1]

array([[25.86073605, 25.56692122, 25.37972711, ..., 24.49387772,
        24.01021861, 23.69854747],
       [25.61181944, 25.41049936, 25.24459452, ..., 24.96101068,
        24.85692889, 24.95629289],
       [25.68724109, 25.37857474, 25.15430572, ..., 24.99756782,
        24.88331497, 24.97290396],
       ...,
       [26.56682992, 26.21038038, 25.93204449, ..., 24.98128735,
        24.773605  , 24.77354078],
       [23.09328893, 22.71486448, 22.47521569, ..., 24.92259083,
        24.66476583, 24.6220505 ],
       [25.47599624, 25.20039922, 24.9858397 , ..., 24.85939203,
        24.65116568, 24.65285036]])

In [ ]:
prediction[:,:,2]

array([[14.9842134 , 15.252905  , 15.45811201, ..., 19.10429369,
        19.76850127, 19.93642214],
       [17.25288097, 17.3571339 , 17.5115589 , ..., 18.61478582,
        18.65686714, 18.57915326],
       [16.82673456, 17.05825597, 17.26152372, ..., 18.57933979,
        18.64832313, 18.58272486],
       ...,
       [14.7476444 , 15.03330736, 15.28125663, ..., 18.4831626 ,
        18.64981722, 18.67716465],
       [19.41317324, 19.6074083 , 19.81687974, ..., 18.69743653,
        18.9260308 , 18.99745775],
       [14.91352677, 15.15561183, 15.3970922 , ..., 18.64086597,
        18.83563176, 18.85806399]])

In [ ]:
prediction[:, :, 0] =  prediction[:, :, 0]
submission['answer'] = prediction[:,:,0].reshape([-1,1])
submission

,num_date_time,answer
0,1 2020-08-25 00,7559.344574
1,1 2020-08-25 01,7521.737850
2,1 2020-08-25 02,7476.054878
3,1 2020-08-25 03,7366.434575
4,1 2020-08-25 04,7348.736409
...,...,...
10075,60 2020-08-31 19,935.635435
10076,60 2020-08-31 20,898.276576
10077,60 2020-08-31 21,853.063113
10078,60 2020-08-31 22,826.041675


In [ ]:
submission.to_csv('sample_submission3.csv', index=False)

In [ ]:
prediction[:, :, 0] =  prediction[:, :, 0]*40
submission['answer'] = prediction[:,:,0].reshape([-1,1])
submission

In [ ]:
submission.to_csv('망한sample_submission.csv', index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(pd.DataFrame(train['전력사용량(kWh)']))
train['	전력사용량(kWh)'] = scaled_train
train['	전력사용량(kWh)']

0         0.461072
1         0.458624
2         0.457017
3         0.453729
4         0.453437
            ...   
122395    0.231936
122396    0.224119
122397    0.201373
122398    0.185982
122399    0.180649
Name: \t전력사용량(kWh), Length: 122400, dtype: float64

In [ ]:
scaled_train2 = scaler.fit_transform(pd.DataFrame(train['기온(°C)']))
train['기온(°C)'] = scaled_train2

In [ ]:
train['기온(°C)']

0         0.257937
1         0.261905
2         0.253968
3         0.238095
4         0.234127
            ...   
122395    0.662698
122396    0.642857
122397    0.642857
122398    0.634921
122399    0.634921
Name: 기온(°C), Length: 122400, dtype: float64